<a href="https://colab.research.google.com/github/juhik24/Chatbot/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing necessary libraries

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()  # For Stemming
!pip install tflearn
import numpy
import tflearn
import tensorflow
import random
import json
import pickle
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127283 sha256=7c8fe007ff36dbafba740aaca97923f29e4cbd0c03890945005abb4b939d75f6
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Loading the intents file

with open("intents.json") as file:
  data = json.load(file)

print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you?', 'Is anyone there?', 'Hello', 'Good day', "What's up", 'how are ya', 'heyy', 'whatsup', '??? ??? ??'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'see you', 'bye bye', 'See you later', 'Goodbye', 'I am Leaving', 'Bye', 'Have a Good day', 'talk to you later', 'ttyl', 'i got to go', 'gtg'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!', 'Come back soon'], 'context_set': ''}, {'tag': 'creator', 'patterns': ['what is the name of your developers', 'what is the name of your creators', 'what is the name of the developers', 'what is the name of the creators', 'who created you', 'your developers', 'your creators', 'who are your developers', 'developers', 'you are made by', 'you are made by whom', 'who created you', 'who create you', 'creators', 'who made you', 'who designed you'], 'responses': ['College Student

In [ ]:
# Trying to load the processed data from a 'data.pickle' file. If the file does not exist, we process the data
# and save it in the file for future use.

try:
  with open("data.pickle", "rb") as f:
    words, labels, training, output = pickle.load(f)
except:
  # If 'data.pickle' file does not exist, we process the data from 'intents.json' file
  words=[]
  labels = []
  docs_x = []
  docs_y = []
  for intent in data["intents"]:
    # Iterate over patterns in each intent and tokenize them
    for pattern in intent["patterns"]:
      wrds = nltk.word_tokenize(pattern)
      words.extend(wrds)
      docs_x.append(wrds)
      docs_y.append(intent["tag"])

    # Add unique tag to the labels list
    if intent["tag"] not in labels:
      labels.append(intent["tag"])

  # Stemming and cleaning the words
  words = [stemmer.stem(w.lower()) for w in words if w != "?"]
  words = sorted(list(set(words)))

  # Sorting labels
  labels = sorted(labels)

  # Initialize training and output lists
  training = []
  output = []

  # Create an empty output list for each label
  out_empty = [0 for _ in range(len(labels))]

  # Create a bag of words for each pattern
  for x, doc in enumerate(docs_x):
    bag = []
    # Stemming and cleaning the words in each pattern
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
      # If the word is in the pattern, add 1 to the bag. Otherwise, add 0.
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)
    # Create an output row for each pattern
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    # Add the bag and output row to the training and output lists
    training.append(bag)
    output.append(output_row)

  # Convert training and output lists to numpy arrays
  training = numpy.array(training)
  output = numpy.array(output)

  # Save words, labels, training, and output in a 'data.pickle' file for future use
  with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)


In [ ]:
print("Training:", training.shape)
print(training[0])
print("Output:", output.shape)
print(output[0])

Training: (405, 243)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Output: (405, 38)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]


In [ ]:
# Reset the default TensorFlow graph
from tensorflow.python.framework import ops
ops.reset_default_graph()

# Define the input layer of the neural network with shape [batch_size, num_features]
net = tflearn.input_data(shape=[None, len(training[0])])

# Add two fully connected hidden layers with 8 nodes each
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)

# Add the output layer with a softmax activation function to get probabilities for each class
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")

# Define the regression layer of the neural network
net = tflearn.regression(net)

# Define a deep neural network with the specified layers
model = tflearn.DNN(net)


# Train the neural network on the training data

#model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
#model.save("model.tflearn")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
# Try to load the saved model from "model.tflearn" file. If the file doesn't exist, train a new model and save it.

try:
  model.load("model.tflearn")
except:
  model = tflearn.DNN(net)
  model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
  model.save("model.tflearn")

Streaming output truncated to the last 5000 lines.
| Adam | epoch: 919 | loss: 0.00000 - acc: 0.9995 -- iter: 136/405
Training Step: 46836  | time: 0.097s
| Adam | epoch: 919 | loss: 0.00000 - acc: 0.9996 -- iter: 144/405
Training Step: 46837  | time: 0.101s
| Adam | epoch: 919 | loss: 0.00000 - acc: 0.9996 -- iter: 152/405
Training Step: 46838  | time: 0.106s
| Adam | epoch: 919 | loss: 0.00000 - acc: 0.9997 -- iter: 160/405
Training Step: 46839  | time: 0.109s
| Adam | epoch: 919 | loss: 0.00000 - acc: 0.9872 -- iter: 168/405
Training Step: 46840  | time: 0.114s
| Adam | epoch: 919 | loss: 0.00000 - acc: 0.9885 -- iter: 176/405
Training Step: 46841  | time: 0.121s
| Adam | epoch: 919 | loss: 0.00000 - acc: 0.9896 -- iter: 184/405
Training Step: 46842  | time: 0.126s
| Adam | epoch: 919 | loss: 0.00000 - acc: 0.9907 -- iter: 192/405
Training Step: 46843  | time: 0.131s
| Adam | epoch: 919 | loss: 0.00000 - acc: 0.9916 -- iter: 200/405
Training Step: 46844  | time: 0.136s
| Adam | epoc

In [ ]:
# This function takes a sentence and a list of words and returns a bag of words representation of the sentence.

def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]
  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1

  return numpy.array(bag)

In [ ]:
from sklearn.metrics import classification_report

# Load the trained model
model.load("model.tflearn")

# Create empty lists to store true and predicted labels
y_true = []
y_pred = []

# Loop over all intents in the data file
for intent in data["intents"]:
    # Loop over all patterns in the intent
    for pattern in intent["patterns"]:
        # Tokenize and stem the pattern
        tokens = nltk.word_tokenize(pattern)
        stemmed = [stemmer.stem(token.lower()) for token in tokens]
        # Convert the pattern to a bag of words
        bow = bag_of_words(pattern, words)
        # Predict the intent for the pattern
        results = model.predict([bow])[0]
        predicted_intent = labels[numpy.argmax(results)]
        # Append the true and predicted labels to the lists
        y_true.append(intent["tag"])
        y_pred.append(predicted_intent)

# Print the classification report, including precision and recall
print(classification_report(y_true, y_pred))


                precision    recall  f1-score   support

     admission       1.00      1.00      1.00         6
       canteen       1.00      1.00      1.00        11
college intake       1.00      1.00      1.00         9
     committee       1.00      1.00      1.00         6
   computerhod       1.00      1.00      1.00         4
        course       1.00      1.00      1.00        27
       creator       1.00      1.00      1.00        16
      director       1.00      1.00      1.00         7
      document       1.00      1.00      1.00        13
        ecehod       1.00      1.00      1.00         4
         event       1.00      1.00      1.00        11
    facilities       1.00      1.00      1.00         5
          fees       1.00      0.96      0.98        23
        floors       1.00      1.00      1.00         7
       goodbye       1.00      1.00      1.00        12
      greeting       1.00      1.00      1.00        10
           hod       1.00      1.00      1.00  

In [ ]:
import sklearn.metrics

# Split data into training and test sets
train_data = data["intents"][:int(len(data["intents"]) * 0.8)]
test_data = data["intents"][int(len(data["intents"]) * 0.8):]

# Make predictions on the test data
y_true = []
y_pred = []
for intent in test_data:
  for pattern in intent["patterns"]:
    x = bag_of_words(pattern, words)
    y_true.append(labels.index(intent["tag"]))
    y_pred.append(numpy.argmax(model.predict([x])))

# Calculate the confusion matrix
cm = sklearn.metrics.confusion_matrix(y_true, y_pred, labels=list(range(len(labels))))

# Print the confusion matrix
print("Confusion matrix:")
print(cm)

# Calculate precision, recall, and F1 score
precision = sklearn.metrics.precision_score(y_true, y_pred, average='weighted')
recall = sklearn.metrics.recall_score(y_true, y_pred, average='weighted')
f1 = sklearn.metrics.f1_score(y_true, y_pred, average='weighted')

# Print the results
print("\nPrecision: ", precision)
print("Recall: ", recall)
print("F1 score: ", f1)

Confusion matrix:
[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  6  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0 12]]

Precision:  1.0
Recall:  1.0
F1 score:  1.0


In [ ]:
print(docs_x[5])
print(docs_y[0])

['What', "'s", 'up']
greeting


In [ ]:
# This chat function takes user input and returns a response from the chatbot.
#For each input the function creates a bag of words representation, feeds it to the trained model, and returns
# a response based on the predicted intent.

def chat():
  print("Start talking with the bot (type quit to stop)!")
  while True:
    inp = input("You: ")
    if inp.lower() == "quit":
      break

    results = model.predict([bag_of_words(inp, words)])
    results_index = numpy.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
      if tg['tag'] == tag:
        responses = tg['responses']

    print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hi
Good to see you again!
You: What is the college fees
For Fee details visit https://iiitranchi.ac.in/fees.aspx
You: What is the syllabus
Timetable is provided directly to the students through their respective Class Representatives.
You: is there any canteen
Our university has canteen with variety of food available
You: and library
There is one huge and spacious library. Timings are 8am to 6pm.
You: What is college timings
College is open 8am-5pm Monday-Friday!
You: what about hostel facilities
For hostel details visit https://iiitranchi.ac.in/facilities.aspx
You: your name
I'm Luna
You: Thank you luna
welcome, anything else i can assist you with?
You: quit
